## 모듈 임포트

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

## 데이터 로드

In [2]:
!wget 'https://bit.ly/3dD5MU9'

import zipfile
with zipfile.ZipFile('3dD5MU9', 'r') as existing_zip:
    existing_zip.extractall('data')

--2021-06-13 12:04:50--  https://bit.ly/3dD5MU9
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1kkF00wW8v0npJ8S2nA7--eMTH3gOL03z [following]
--2021-06-13 12:04:50--  https://drive.google.com/uc?export=download&id=1kkF00wW8v0npJ8S2nA7--eMTH3gOL03z
Resolving drive.google.com (drive.google.com)... 108.177.111.102, 108.177.111.101, 108.177.111.113, ...
Connecting to drive.google.com (drive.google.com)|108.177.111.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/advg3vvi29oog5mdeovujr45v5vglfo8/1623585825000/00192245294648390361/*/1kkF00wW8v0npJ8S2nA7--eMTH3gOL03z?e=download [following]
--2021-06-13 12:04:53--  https://doc-14-94-docs.googleusercontent.com/docs/secur

In [3]:
energy = pd.read_csv('data/energy.csv')
dangjin_fcst = pd.read_csv('./data/dangjin_fcst_data.csv')
ulsan_fcst = pd.read_csv('./data/ulsan_fcst_data.csv')

## 데이터 병합 
- 가장 나중의 예측 데이터가 가장 좋다고 가정.

In [4]:
def to_date(x):
    return timedelta(hours=x)

In [5]:
def generate_df(df_):
    df = df_.copy()

    df['Forecast_time'] = pd.to_datetime(df['Forecast time'])

    # 20 시
    a = df[df["forecast"] == 4.0]
    a = a[a["Forecast time"].apply(lambda x: "20:00:00" in x)]
    a.loc[:, 'Forecast_time'] = a.loc[:, 'Forecast_time'] + a.loc[:, 'forecast'].map(to_date)

    # 23 시
    b = df[df["forecast"] <= 22]
    b = b[b["Forecast time"].apply(lambda x: "23:00:00" in x)]
    b.loc[:, 'Forecast_time'] = b.loc[:, 'Forecast_time'] + b.loc[:, 'forecast'].map(to_date)

    # 병합
    c = pd.concat([a, b])
    print(f"20시 사용 데이터 길이 : {len(a)}")
    print(f"23시 사용 데이터 길이 : {len(b)}")
    print(f"합친 데이터 길이 : {len(c)}")
    print()

    # 정렬
    c.sort_values(by=['Forecast_time'], inplace=True)
    c = c[['Forecast_time', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud']]

    return c

In [6]:
dangjin_filled = generate_df(dangjin_fcst)
ulsan_filled = generate_df(ulsan_fcst)

20시 사용 데이터 길이 : 1096
23시 사용 데이터 길이 : 7672
합친 데이터 길이 : 8768

20시 사용 데이터 길이 : 1096
23시 사용 데이터 길이 : 7672
합친 데이터 길이 : 8768



## 보간

In [7]:
def interpolate_df(df, method='linear'):
    new_df = pd.DataFrame()
    new_df['Forecast_time'] = pd.date_range(start=df['Forecast_time'].iloc[0], end=df['Forecast_time'].iloc[-1], freq='H')
    new_df = pd.merge(new_df, df, on='Forecast_time', how='outer')
    return new_df.interpolate(method=method)

In [8]:
dangjin_interpolated = interpolate_df(dangjin_filled, method='linear')
ulsan_interpolated = interpolate_df(ulsan_filled, method='linear')

## 학습 데이터 전처리

In [9]:
def train_datast(energy_df, fcst_df, target):
    # 일기 예보 있는 날짜만 선택
    energy = energy_df.loc[24:]
    energy.index = range(energy.shape[0])
    
    # 발전량 데이터가 있는 날짜만 선택
    fcst = fcst_df.loc[:25608-1]
    fcst.index = range(fcst.shape[0])
    
    # 발전량과 일기예보 연결
    concat_df = pd.concat([energy, fcst], axis=1)
    
    # 예보 시간 및 날짜 정보 feature로 추가
    concat_df['date'] = concat_df['Forecast_time'].str.split(' ').str[0]
    concat_df['hour'] = concat_df['Forecast_time'].str.split(' ').str[1].str.split(':').str[0].astype(int)
    
    concat_df['year'] = concat_df['date'].str.split('-').str[0].astype(int)
    concat_df['month'] = concat_df['date'].str.split('-').str[1].astype(int)
    concat_df['day'] = concat_df['date'].str.split('-').str[2].astype(int)
    
    # 예보 시간, 날짜, 기상 예보 및 발전량 선택
    feature_df = concat_df[['year', 'month', 'day', 'hour', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud', target]]
    
    # 마지막 30일을 검증데이터셋으로 나머지를 학습 데이터셋으로 선택
    train_df = feature_df.iloc[:-24*90]
    val_df = feature_df.iloc[-24*90:]
    
    # 발전량이 0인 데이터를 제외
    #train_df = train_df[train_df[target]!=0]
    
    train_x = train_df.loc[:, 'year':'Cloud'].to_numpy()
    train_y = train_df[target].to_numpy()
    val_x = val_df.loc[:, 'year':'Cloud'].to_numpy()
    val_y = val_df[target].to_numpy()
    
    return train_x, train_y, val_x, val_y

## bayesian optimization 
---
- bayesian optimization 를 이용하여 하이퍼 파라미터 튜닝
- lgbm만 적용.

In [10]:
# bayesian optimization 패키지 설치
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11686 sha256=13a90544c739290a7e66731b876402ba580036175f0326b3cd917bcb864bb332
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [11]:
!pip install Catboost
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

     |████████████████████████████████| 69.2MB 62kB/s 


In [12]:
dangjin_interpolated['Forecast_time'] = dangjin_interpolated['Forecast_time'].astype('str')
ulsan_interpolated['Forecast_time'] = ulsan_interpolated['Forecast_time'].astype('str')

In [13]:
energy = energy.fillna(0)

## 당진 수상태양광 예측 모델 학습

In [ ]:
bayesian_params = {
    'max_depth': (5, 20), 
    'num_leaves': (24, 64), 
    'min_child_samples': (10, 200), 
    'min_child_weight':(1, 50),
    'subsample':(0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha': (0.01, 50) 
}

def lgb_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                colsample_bytree,max_bin, reg_lambda, reg_alpha):
    params = {
        "n_estimators":1000, "learning_rate":0.01,
        'max_depth': int(round(max_depth)), #  호출 시 실수형 값이 들어오므로 정수형 하이퍼 파라미터는 정수형으로 변경 
        'num_leaves': int(round(num_leaves)), 
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample': max(min(subsample, 1), 0), 
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'max_bin':  max(int(round(max_bin)),10),
        'reg_lambda': max(reg_lambda,0),
        'reg_alpha': max(reg_alpha, 0),
        "metric" : 'rmse'
    }
    cv_result = lgb.cv(params, train_data, nfold=5, seed=0, verbose_eval =200,stratified=False)
    return (-1.0 * np.array(cv_result['rmse-mean'])).max()

train_x, train_y, val_x, val_y = train_datast(energy, dangjin_interpolated, target='dangjin_floating')
train_data = lgb.Dataset(train_x, train_y)
lgbBO = BayesianOptimization(lgb_eval, bayesian_params, random_state=42)


lgbBO.maximize(init_points=5, n_iter=25)
####################################################
# dictionary에 있는 target값을 모두 추출
target_list = []
for result in lgbBO.res:
    target = result['target']
    target_list.append(target)
print(target_list)
# 가장 큰 target 값을 가지는 순번(index)를 추출
print('maximum target index:', np.argmax(np.array(target_list)))

#################################################################
# 가장 큰 target값을 가지는 index값을 기준으로 res에서 해당 parameter 추출. 
max_dict = lgbBO.res[np.argmax(np.array(target_list))]
print(max_dict)

## K-fold dataset

In [16]:
from sklearn.model_selection import StratifiedKFold
import random

In [17]:
def kfold_datast(energy_df, fcst_df, target):
    # 일기 예보 있는 날짜만 선택
    energy = energy_df.loc[24:]
    energy.index = range(energy.shape[0])
    
    # 발전량 데이터가 있는 날짜만 선택
    fcst = fcst_df.loc[:25608-1]
    fcst.index = range(fcst.shape[0])
    
    # 발전량과 일기예보 연결
    concat_df = pd.concat([energy, fcst], axis=1)
    
    # 예보 시간 및 날짜 정보 feature로 추가
    concat_df['date'] = concat_df['Forecast_time'].str.split(' ').str[0]
    concat_df['hour'] = concat_df['Forecast_time'].str.split(' ').str[1].str.split(':').str[0].astype(int)
    
    concat_df['year'] = concat_df['date'].str.split('-').str[0].astype(int)
    concat_df['month'] = concat_df['date'].str.split('-').str[1].astype(int)
    concat_df['day'] = concat_df['date'].str.split('-').str[2].astype(int)
    
    # 예보 시간, 날짜, 기상 예보 및 발전량 선택
    feature_df = concat_df[['year', 'month', 'day', 'hour', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud', target]]
    return feature_df

In [ ]:
random.seed(42)
lgb1_models={}
target = 'dangjin_floating'

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds=[]
train = kfold_datast (energy, dangjin_interpolated, target=target)


for train_idx, valid_idx in skf.split(train, train['dangjin_floating']):
    folds.append((train_idx, valid_idx))

for fold in range(10):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    train_x, val_x, train_y, val_y = train.drop([target],axis=1).iloc[train_idx].values, train.drop([target],axis=1).iloc[valid_idx].values,\
                                         train[target][train_idx].values, train[target][valid_idx].values 
    lgb = LGBMRegressor(n_estimators=5000 ,random_seed=42, learning_rate=0.005,depth = 10,loss_function='MultiRMSE',
                                        colsample_bytree =  1,max_depth = 17, min_child_samples = 17, min_child_weight = 14, num_leaves = 63, reg_alpha=49.75855787791481, reg_lambda= 0.026935271560602856, subsample= 1.0)
    lgb.fit(train_x, train_y,
                    eval_set=[(train_x, train_y), (val_x, val_y)],
                    early_stopping_rounds=50,verbose=100)
    lgb1_models[fold]=lgb
    print(f'================================================================================\n\n')
    #2364

### 모델저장

In [47]:
from sklearn.externals import joblib
for i in range(10):
    joblib.dump(lgb1_models[i],'./dangjin_floating/dangjin_floating_{0}.pkl'.format(i+1))

### cat boost 사용 x 

In [ ]:
'''train_x, train_y, val_x, val_y = train_datast(energy, dangjin_interpolated, target='dangjin_floating')
cat1 = CatBoostRegressor(n_estimators=5000 ,random_seed=42, learning_rate=0.005,depth = 10,loss_function='MultiRMSE')
cat1.fit(train_x, train_y,
                  eval_set=[(train_x, train_y), (val_x, val_y)],
                  early_stopping_rounds=100,verbose=100)'''

### lgbm 사용

In [ ]:
train_x, train_y, val_x, val_y = train_datast(energy, dangjin_interpolated, target='dangjin_floating')
dangjin_floating_model = LGBMRegressor(n_estimators=5000 ,random_seed=42, learning_rate=0.005,depth = 10,loss_function='MultiRMSE',
                                       colsample_bytree =  1,max_depth = 17, min_child_samples = 17, min_child_weight = 14, num_leaves = 63, reg_alpha=49.75855787791481, reg_lambda= 0.026935271560602856, subsample= 1.0)
dangjin_floating_model.fit(train_x, train_y,
                  eval_set=[(train_x, train_y), (val_x, val_y)],
                  early_stopping_rounds=100,verbose=100)

Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 17468.2	valid_1's l2: 12314.7
[200]	training's l2: 9505.46	valid_1's l2: 7518.53
[300]	training's l2: 6160.28	valid_1's l2: 5552.15
[400]	training's l2: 4621.98	valid_1's l2: 4734.6
[500]	training's l2: 3850.62	valid_1's l2: 4397.85
[600]	training's l2: 3360.87	valid_1's l2: 4265.15
[700]	training's l2: 3025.62	valid_1's l2: 4199.41
[800]	training's l2: 2769.75	valid_1's l2: 4179.53
[900]	training's l2: 2594.36	valid_1's l2: 4162.22
[1000]	training's l2: 2454.82	valid_1's l2: 4151.01
[1100]	training's l2: 2326.93	valid_1's l2: 4141.93
[1200]	training's l2: 2230.02	valid_1's l2: 4123.69
[1300]	training's l2: 2152.63	valid_1's l2: 4103.76
[1400]	training's l2: 2065.5	valid_1's l2: 4105.27
Early stopping, best iteration is:
[1309]	training's l2: 2144.3	valid_1's l2: 4102.56


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1,
              depth=10, importance_type='split', learning_rate=0.005,
              loss_function='MultiRMSE', max_depth=17, min_child_samples=17,
              min_child_weight=14, min_split_gain=0.0, n_estimators=5000,
              n_jobs=-1, num_leaves=63, objective=None, random_seed=42,
              random_state=None, reg_alpha=49.75855787791481,
              reg_lambda=0.026935271560602856, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

## 당진 자재 창고 태양광 예측 모델 학습

In [ ]:
bayesian_params = {
    'max_depth': (5, 20), 
    'num_leaves': (24, 64), 
    'min_child_samples': (10, 200), 
    'min_child_weight':(1, 50),
    'subsample':(0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha': (0.01, 50) 
}

def lgb_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                colsample_bytree,max_bin, reg_lambda, reg_alpha):
    params = {
        "n_estimators":1000, "learning_rate":0.01,
        'max_depth': int(round(max_depth)), #  호출 시 실수형 값이 들어오므로 정수형 하이퍼 파라미터는 정수형으로 변경 
        'num_leaves': int(round(num_leaves)), 
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample': max(min(subsample, 1), 0), 
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'max_bin':  max(int(round(max_bin)),10),
        'reg_lambda': max(reg_lambda,0),
        'reg_alpha': max(reg_alpha, 0),
        "metric" : 'rmse'
    }
    cv_result = lgb.cv(params, train_data, nfold=5, seed=0, verbose_eval =200,stratified=False)
    return (-1.0 * np.array(cv_result['rmse-mean'])).max()

train_x, train_y, val_x, val_y = train_datast(energy, dangjin_interpolated, target='dangjin_warehouse')
train_data = lgb.Dataset(train_x, train_y)
lgbBO = BayesianOptimization(lgb_eval, bayesian_params, random_state=42)




lgbBO.maximize(init_points=5, n_iter=25)
####################################################
# dictionary에 있는 target값을 모두 추출
target_list = []
for result in lgbBO.res:
    target = result['target']
    target_list.append(target)
print(target_list)
# 가장 큰 target 값을 가지는 순번(index)를 추출
print('maximum target index:', np.argmax(np.array(target_list)))

#################################################################
# 가장 큰 target값을 가지는 index값을 기준으로 res에서 해당 parameter 추출. 
max_dict = lgbBO.res[np.argmax(np.array(target_list))]
print(max_dict)

In [ ]:
random.seed(42)
cat2_models={}
target = 'dangjin_warehouse'

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds=[]
train = kfold_datast (energy, dangjin_interpolated, target=target)


for train_idx, valid_idx in skf.split(train, train['dangjin_warehouse']):
    folds.append((train_idx, valid_idx))

for fold in range(10):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    train_x, val_x, train_y, val_y = train.drop([target],axis=1).iloc[train_idx].values, train.drop([target],axis=1).iloc[valid_idx].values,\
                                         train[target][train_idx].values, train[target][valid_idx].values 
    cat = CatBoostRegressor(n_estimators=5000 ,random_seed=42, learning_rate=0.005,depth = 10,loss_function='MultiRMSE')
    cat.fit(train_x, train_y,
                    eval_set=[(train_x, train_y), (val_x, val_y)],
                    early_stopping_rounds=100,verbose=100)
    cat2_models[fold]=cat
    print(f'================================================================================\n\n')

### 모델 저장

In [48]:
from sklearn.externals import joblib
for i in range(10):
    joblib.dump(cat2_models[i],'./dangjin_warehouse/dangjin_warehouse{0}.pkl'.format(i+1))

### cat 사용

In [ ]:
train_x, train_y, val_x, val_y = train_datast(energy, dangjin_interpolated, target='dangjin_warehouse')
cat2 = CatBoostRegressor(n_estimators=5000 ,random_seed=42, learning_rate=0.005,depth = 10,loss_function='MultiRMSE')
cat2.fit(train_x, train_y,
                  eval_set=[(train_x, train_y), (val_x, val_y)],
                  early_stopping_rounds=100,verbose=100)

0:	learn: 147.2777493	test: 147.2777493	test1: 127.8906322	best: 127.8906322 (0)	total: 75.1ms	remaining: 6m 15s
100:	learn: 103.6944464	test: 103.6944464	test1: 91.7248245	best: 91.7248245 (100)	total: 2.45s	remaining: 1m 59s
200:	learn: 79.2017736	test: 79.2017736	test1: 72.3140566	best: 72.3140566 (200)	total: 4.76s	remaining: 1m 53s
300:	learn: 65.9537681	test: 65.9537681	test1: 62.8602773	best: 62.8602773 (300)	total: 7.06s	remaining: 1m 50s
400:	learn: 58.8730529	test: 58.8730529	test1: 58.3327560	best: 58.3327560 (400)	total: 9.37s	remaining: 1m 47s
500:	learn: 54.7668253	test: 54.7668253	test1: 56.4786361	best: 56.4786361 (500)	total: 11.7s	remaining: 1m 44s
600:	learn: 52.1889608	test: 52.1889608	test1: 55.4925947	best: 55.4925947 (600)	total: 14s	remaining: 1m 42s
700:	learn: 50.3383994	test: 50.3383994	test1: 54.9446819	best: 54.9446819 (700)	total: 16.3s	remaining: 1m 39s
800:	learn: 48.9941662	test: 48.9941662	test1: 54.7032289	best: 54.6956555 (792)	total: 18.6s	remaining


### lgbm 사용 X basian optimizer 했지만 성능 하락

In [ ]:
{'colsample_bytree': 0.7840043724584478, 'max_bin': 109.42082906265013, 'max_depth': 11.939485103471496, 'min_child_samples': 47.27436173622021, 'min_child_weight': 44.12615815161272, 'num_leaves': 61.168181706089655, 'reg_alpha': 7.967786769707356, 'reg_lambda': 1.4818125424587414, 'subsample': 0.835744966051806}}

In [ ]:
train_x, train_y, val_x, val_y = train_datast(energy, dangjin_interpolated, target='dangjin_warehouse')
dangjin_warehouse_model = LGBMRegressor(n_estimators=5000 ,random_seed=42, learning_rate=0.005,depth = 8,loss_function='MultiRMSE',
                                       colsample_bytree =  1,max_depth = 17, min_child_samples = 17, min_child_weight = 14, num_leaves = 63, reg_alpha=49.75855787791481, reg_lambda= 0.026935271560602856, subsample= 1.0)
dangjin_warehouse_model.fit(train_x, train_y,
                  eval_set=[(train_x, train_y), (val_x, val_y)],
                  early_stopping_rounds=100,verbose=100)

Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 10057.3	valid_1's l2: 8089.53
[200]	training's l2: 5471.98	valid_1's l2: 4970.63
[300]	training's l2: 3539.38	valid_1's l2: 3813.67
[400]	training's l2: 2674.14	valid_1's l2: 3368.68
[500]	training's l2: 2229.72	valid_1's l2: 3195.32
[600]	training's l2: 1963	valid_1's l2: 3129.7
[700]	training's l2: 1785.8	valid_1's l2: 3105.45
[800]	training's l2: 1651.59	valid_1's l2: 3057.7
[900]	training's l2: 1547.15	valid_1's l2: 3032.77
[1000]	training's l2: 1460.65	valid_1's l2: 3021.56
[1100]	training's l2: 1380.61	valid_1's l2: 3024.25
Early stopping, best iteration is:
[1051]	training's l2: 1417.22	valid_1's l2: 3020.03


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1,
              depth=8, importance_type='split', learning_rate=0.005,
              loss_function='MultiRMSE', max_depth=17, min_child_samples=17,
              min_child_weight=14, min_split_gain=0.0, n_estimators=5000,
              n_jobs=-1, num_leaves=63, objective=None, random_seed=42,
              random_state=None, reg_alpha=49.75855787791481,
              reg_lambda=0.026935271560602856, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

## 당진 태양광 예측 모델 학습

In [ ]:
bayesian_params = {
    'max_depth': (5, 20), 
    'num_leaves': (24, 64), 
    'min_child_samples': (10, 200), 
    'min_child_weight':(1, 50),
    'subsample':(0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha': (0.01, 50) 
}

def lgb_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                colsample_bytree,max_bin, reg_lambda, reg_alpha):
    params = {
        "n_estimators":1000, "learning_rate":0.01,
        'max_depth': int(round(max_depth)), #  호출 시 실수형 값이 들어오므로 정수형 하이퍼 파라미터는 정수형으로 변경 
        'num_leaves': int(round(num_leaves)), 
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample': max(min(subsample, 1), 0), 
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'max_bin':  max(int(round(max_bin)),10),
        'reg_lambda': max(reg_lambda,0),
        'reg_alpha': max(reg_alpha, 0),
        "metric" : 'rmse'
    }
    cv_result = lgb.cv(params, train_data, nfold=5, seed=0, verbose_eval =200,stratified=False)
    return (-1.0 * np.array(cv_result['rmse-mean'])).max()

train_x, train_y, val_x, val_y = train_datast(energy, dangjin_interpolated, target='dangjin')
train_data = lgb.Dataset(train_x, train_y)
lgbBO = BayesianOptimization(lgb_eval, bayesian_params, random_state=42)



lgbBO.maximize(init_points=5, n_iter=25)
####################################################
# dictionary에 있는 target값을 모두 추출
target_list = []
for result in lgbBO.res:
    target = result['target']
    target_list.append(target)
print(target_list)
# 가장 큰 target 값을 가지는 순번(index)를 추출
print('maximum target index:', np.argmax(np.array(target_list)))

#################################################################
# 가장 큰 target값을 가지는 index값을 기준으로 res에서 해당 parameter 추출. 
max_dict = lgbBO.res[np.argmax(np.array(target_list))]
print(max_dict)

In [ ]:
random.seed(42)
cat3_models={}
target = 'dangjin'

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds=[]
train = kfold_datast (energy, dangjin_interpolated, target=target)


for train_idx, valid_idx in skf.split(train, train['dangjin']):
    folds.append((train_idx, valid_idx))

for fold in range(10):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    train_x, val_x, train_y, val_y = train.drop([target],axis=1).iloc[train_idx].values, train.drop([target],axis=1).iloc[valid_idx].values,\
                                         train[target][train_idx].values, train[target][valid_idx].values 
    cat = CatBoostRegressor(n_estimators=5000 ,random_seed=42, learning_rate=0.005,depth = 10,loss_function='MultiRMSE')
    cat.fit(train_x, train_y,
                    eval_set=[(train_x, train_y), (val_x, val_y)],
                    early_stopping_rounds=50,verbose=100)
    cat3_models[fold]=cat
    print(f'================================================================================\n\n')

### 모델 저장

In [49]:
from sklearn.externals import joblib
for i in range(10):
    joblib.dump(cat3_models[i],'./dangjin/dangjin{0}.pkl'.format(i+1))

### cat 사용

In [ ]:
train_x, train_y, val_x, val_y = train_datast(energy, dangjin_interpolated, target='dangjin')
cat3 = CatBoostRegressor(n_estimators=5000 ,random_seed=42, learning_rate=0.005,depth = 10,loss_function='MultiRMSE')
cat3.fit(train_x, train_y,
                  eval_set=[(train_x, train_y), (val_x, val_y)],
                  early_stopping_rounds=100,verbose=100)

0:	learn: 221.6627612	test: 221.6627612	test1: 195.8197076	best: 195.8197076 (0)	total: 24.2ms	remaining: 2m 1s
100:	learn: 154.3536660	test: 154.3536660	test1: 140.5623408	best: 140.5623408 (100)	total: 2.38s	remaining: 1m 55s
200:	learn: 116.2758635	test: 116.2758635	test1: 111.7606646	best: 111.7606646 (200)	total: 4.71s	remaining: 1m 52s
300:	learn: 95.6359842	test: 95.6359842	test1: 97.2736954	best: 97.2736954 (300)	total: 7s	remaining: 1m 49s
400:	learn: 84.8683633	test: 84.8683633	test1: 90.2021170	best: 90.2021170 (400)	total: 9.29s	remaining: 1m 46s
500:	learn: 78.9467163	test: 78.9467163	test1: 86.7140935	best: 86.7140935 (500)	total: 11.6s	remaining: 1m 44s
600:	learn: 75.4799602	test: 75.4799602	test1: 85.0015122	best: 85.0015122 (600)	total: 13.9s	remaining: 1m 41s
700:	learn: 73.1411108	test: 73.1411108	test1: 84.1336129	best: 84.1215786 (699)	total: 16.2s	remaining: 1m 39s
800:	learn: 71.4765393	test: 71.4765393	test1: 83.5197950	best: 83.5197950 (800)	total: 18.5s	remai


### lgbm 사용 X basian optimizer 했지만 성능 하락

In [ ]:
{'target': -67.37835160276464, 'params': {'colsample_bytree': 0.8803371255183132, 'max_bin': 366.3312849753787, 'max_depth': 14.426679148899519, 'min_child_samples': 24.334988996735756, 'min_child_weight': 14.85309783894488, 'num_leaves': 41.115245176939176, 'reg_alpha': 1.1811080746620453, 'reg_lambda': 9.672295351116277, 'subsample': 0.6051769440265253}}

In [ ]:
train_x, train_y, val_x, val_y = train_datast(energy, dangjin_interpolated, target='dangjin')
dangjin_model = LGBMRegressor(n_estimators=5000 ,random_seed=42, learning_rate=0.005,depth = 8,loss_function='MultiRMSE',
                                       colsample_bytree =  0.7840043724584478,max_bin= 109,num_leaves = 61,
                                        max_depth = 12, min_child_samples = 47, min_child_weight = 44, reg_alpha=7.96, reg_lambda= 1.48, subsample= 0.835744966051806)
dangjin_model.fit(train_x, train_y,
                  eval_set=[(train_x, train_y), (val_x, val_y)],
                  early_stopping_rounds=100,verbose=100)

Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 23972.9	valid_1's l2: 20038.7
[200]	training's l2: 13505.7	valid_1's l2: 12722.1
[300]	training's l2: 8680.92	valid_1's l2: 9453.09
[400]	training's l2: 6494.53	valid_1's l2: 8076.26
[500]	training's l2: 5396.67	valid_1's l2: 7532.7
[600]	training's l2: 4788.04	valid_1's l2: 7312.69
[700]	training's l2: 4394.52	valid_1's l2: 7184.58
[800]	training's l2: 4115.53	valid_1's l2: 7123.25
[900]	training's l2: 3903.11	valid_1's l2: 7085.49
[1000]	training's l2: 3718.13	valid_1's l2: 7095.42
Early stopping, best iteration is:
[912]	training's l2: 3878.86	valid_1's l2: 7080.32


LGBMRegressor(boosting_type='gbdt', class_weight=None,
              colsample_bytree=0.7840043724584478, depth=8,
              importance_type='split', learning_rate=0.005,
              loss_function='MultiRMSE', max_bin=109, max_depth=12,
              min_child_samples=47, min_child_weight=44, min_split_gain=0.0,
              n_estimators=5000, n_jobs=-1, num_leaves=61, objective=None,
              random_seed=42, random_state=None, reg_alpha=7.96,
              reg_lambda=1.48, silent=True, subsample=0.835744966051806,
              subsample_for_bin=200000, subsample_freq=0)

## 울산 태양광 예측 모델 학습

In [ ]:
bayesian_params = {
    'max_depth': (5, 20), 
    'num_leaves': (24, 64), 
    'min_child_samples': (10, 200), 
    'min_child_weight':(1, 50),
    'subsample':(0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha': (0.01, 50) 
}

def lgb_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                colsample_bytree,max_bin, reg_lambda, reg_alpha):
    params = {
        "n_estimators":1000, "learning_rate":0.01,
        'max_depth': int(round(max_depth)), #  호출 시 실수형 값이 들어오므로 정수형 하이퍼 파라미터는 정수형으로 변경 
        'num_leaves': int(round(num_leaves)), 
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample': max(min(subsample, 1), 0), 
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'max_bin':  max(int(round(max_bin)),10),
        'reg_lambda': max(reg_lambda,0),
        'reg_alpha': max(reg_alpha, 0),
        "metric" : 'rmse'
    }
    cv_result = lgb.cv(params, train_data, nfold=5, seed=0, verbose_eval =200,stratified=False)
    return (-1.0 * np.array(cv_result['rmse-mean'])).max()

train_x, train_y, val_x, val_y = train_datast(energy, ulsan_interpolated, target='ulsan')
train_data = lgb.Dataset(train_x, train_y)

lgbB1 = BayesianOptimization(lgb_eval, bayesian_params, random_state=42)


lgbB1.maximize(init_points=5, n_iter=25)
####################################################
# dictionary에 있는 target값을 모두 추출
target_list = []
for result in lgbB1.res:
    target = result['target']
    target_list.append(target)
print(target_list)
# 가장 큰 target 값을 가지는 순번(index)를 추출
print('maximum target index:', np.argmax(np.array(target_list)))

#################################################################
# 가장 큰 target값을 가지는 index값을 기준으로 res에서 해당 parameter 추출. 
max_dict = lgbB1.res[np.argmax(np.array(target_list))]
print(max_dict)

In [ ]:
random.seed(42)
cat4_models={}
target = 'ulsan'

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds=[]
train = kfold_datast (energy, dangjin_interpolated, target=target)


for train_idx, valid_idx in skf.split(train, train['ulsan']):
    folds.append((train_idx, valid_idx))

for fold in range(10):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    train_x, val_x, train_y, val_y = train.drop([target],axis=1).iloc[train_idx].values, train.drop([target],axis=1).iloc[valid_idx].values,\
                                         train[target][train_idx].values, train[target][valid_idx].values 
    cat = CatBoostRegressor(n_estimators=5000 ,random_seed=42, learning_rate=0.005,depth = 10,loss_function='MultiRMSE')
    cat.fit(train_x, train_y,
                    eval_set=[(train_x, train_y), (val_x, val_y)],
                    early_stopping_rounds=50,verbose=100)
    cat4_models[fold]=cat
    print(f'================================================================================\n\n')

### 모델 저장


In [50]:
from sklearn.externals import joblib
for i in range(10):
    joblib.dump(cat4_models[i],'./ulsan/ulsan_{0}.pkl'.format(i+1))

## cat사용

In [ ]:
train_x, train_y, val_x, val_y = train_datast(energy, ulsan_interpolated, target='ulsan')
cat4 = CatBoostRegressor(n_estimators=5000 ,random_seed=42, learning_rate=0.005,depth = 10,loss_function='MultiRMSE',)
cat4.fit(train_x, train_y,
                  eval_set=[(train_x, train_y), (val_x, val_y)],
                  early_stopping_rounds=100,verbose=100)

0:	learn: 100.9963701	test: 100.9963701	test1: 96.1193850	best: 96.1193850 (0)	total: 24.2ms	remaining: 2m 1s
100:	learn: 70.2557123	test: 70.2557123	test1: 65.9418300	best: 65.9418300 (100)	total: 2.3s	remaining: 1m 51s
200:	learn: 52.9166766	test: 52.9166766	test1: 48.9462749	best: 48.9462749 (200)	total: 4.6s	remaining: 1m 49s
300:	learn: 43.6255030	test: 43.6255030	test1: 39.6344758	best: 39.6344758 (300)	total: 6.9s	remaining: 1m 47s
400:	learn: 38.5923310	test: 38.5923310	test1: 34.3495232	best: 34.3495232 (400)	total: 9.2s	remaining: 1m 45s
500:	learn: 35.7032955	test: 35.7032955	test1: 31.5021386	best: 31.5021386 (500)	total: 11.5s	remaining: 1m 43s
600:	learn: 33.9247382	test: 33.9247382	test1: 29.8900932	best: 29.8900932 (600)	total: 13.8s	remaining: 1m 40s
700:	learn: 32.6665984	test: 32.6665984	test1: 28.8877333	best: 28.8877333 (700)	total: 16.1s	remaining: 1m 38s
800:	learn: 31.7384537	test: 31.7384537	test1: 28.2436411	best: 28.2436411 (800)	total: 18.3s	remaining: 1m 36


### lgbm 사용 X basian optimizer 했지만 성능 하락

In [ ]:
train_x, train_y, val_x, val_y = train_datast(energy, ulsan_interpolated, target='ulsan')
ulsan_model = LGBMRegressor(n_estimators=5000 ,random_seed=42, learning_rate=0.005,depth = 8,loss_function='MultiRMSE',
                                       colsample_bytree =  1,max_depth = 17, min_child_samples = 17, min_child_weight = 14, num_leaves = 63, reg_alpha=49.75855787791481, reg_lambda= 0.026935271560602856, subsample= 1.0)
ulsan_model.fit(train_x, train_y,
                  eval_set=[(train_x, train_y), (val_x, val_y)],
                  early_stopping_rounds=100,verbose=100)

## privat 제출 

In [26]:
import pandas as pd
import urllib
import urllib.request
import json

In [28]:
def private(nx,ny,fcst_day,date):
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst'


    queryParams = '?' + urllib.parse.urlencode(
        {
            urllib.parse.quote_plus('ServiceKey') : 'k4ddN+RdAKoBExDTI0PS+2M3QtLVZZkW7Bd8in1h4j5sgfrd2cyVRmYieYHcJNTTnPd+b7X8epYmZS5Ngj18Ww==', # key를 바로 입력해도 됩니다.
            urllib.parse.quote_plus('numOfRows') : '113', # 총 14개의 항목을 3시간 단위로 순차적으로 불러옵니다. 다음날 24시간예보에 필요한 만큼만 가져왔습니다.
            urllib.parse.quote_plus('dataType') : 'JSON', # JSON, XML 두가지 포멧을 제공합니다.
            urllib.parse.quote_plus('base_date') : fcst_day, # 예보 받을 날짜를 입력합니다. 최근 1일간의 자료만 제공합니다.
            urllib.parse.quote_plus('base_time') : '2000', # 예보 시간을 입력합니다. 2시부터 시작하여 3시간 단위로 입력 가능합니다.
            urllib.parse.quote_plus('nx') : nx, # 울산 태양광 발전소 x 좌표입니다. '기상청18_동네예보 조회서비스_오픈API활용가이드.zip'에 포함 된 excel파일을 통해 확인 가능합니다.
            urllib.parse.quote_plus('ny') : ny # 울산 태양광 발전소 y 좌표입니다. '기상청18_동네예보 조회서비스_오픈API활용가이드.zip'에 포함 된 excel파일을 통해 확인 가능합니다.
        }
    )

    response = urllib.request.urlopen(url + queryParams).read()
    response = json.loads(response)
    fcst_df = pd.DataFrame()

    fcst_df['Forecast_time'] = [f'{date} {hour}:00' for hour in range(24)]
    row_idx = 0

    for i, data in enumerate(response['response']['body']['items']['item']):
        if i > 19:
            if data['category']=='REH':
                fcst_df.loc[row_idx, 'Humidity'] = float(data['fcstValue'])
                print('category:Humidity,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
            elif data['category']=='T3H':
                fcst_df.loc[row_idx, 'Temperature'] = float(data['fcstValue'])
                print('category:Temperature,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
            elif data['category']=='SKY':
                fcst_df.loc[row_idx, 'Cloud'] = float(data['fcstValue'])
                print('category:Cloud,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
            elif data['category']=='VEC':
                fcst_df.loc[row_idx, 'WindDirection'] = float(data['fcstValue'])
                print('category:WindDirection,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
            elif data['category']=='WSD':
                fcst_df.loc[row_idx, 'WindSpeed'] = float(data['fcstValue'])
                print('category:WindSpeed,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'], '\n')
                row_idx+=3

    fcst_df = fcst_df.interpolate()
    fcst_df = fcst_df.iloc[:24]
    return fcst_df

In [29]:
#dangjin = 53 , 44
#ulsan = 102, 83
#nx, ny = '53','44'

fcst_day='20210613'# 오늘 날짜
date = '2021-06-14'  #내일 날짜
day = 6 #6월 8일  = 1
ulsan = private('102' ,'83' ,fcst_day,date)
dangjin = private('53','44' ,fcst_day,date)


category:Humidity, REH baseTime: 2000 , fcstTime: 0600 , fcstValue: 90
category:Cloud, SKY baseTime: 2000 , fcstTime: 0600 , fcstValue: 4
category:Temperature, T3H baseTime: 2000 , fcstTime: 0600 , fcstValue: 22
category:WindDirection, VEC baseTime: 2000 , fcstTime: 0600 , fcstValue: 40
category:WindSpeed, WSD baseTime: 2000 , fcstTime: 0600 , fcstValue: 1.6 

category:Humidity, REH baseTime: 2000 , fcstTime: 0900 , fcstValue: 80
category:Cloud, SKY baseTime: 2000 , fcstTime: 0900 , fcstValue: 4
category:Temperature, T3H baseTime: 2000 , fcstTime: 0900 , fcstValue: 23
category:WindDirection, VEC baseTime: 2000 , fcstTime: 0900 , fcstValue: 39
category:WindSpeed, WSD baseTime: 2000 , fcstTime: 0900 , fcstValue: 2.1 

category:Humidity, REH baseTime: 2000 , fcstTime: 1200 , fcstValue: 70
category:Cloud, SKY baseTime: 2000 , fcstTime: 1200 , fcstValue: 4
category:Temperature, T3H baseTime: 2000 , fcstTime: 1200 , fcstValue: 25
category:WindDirection, VEC baseTime: 2000 , fcstTime: 1200 , 

In [32]:
def test_datast(fcst_df):
    start = '2021-06-14 0:00'
    end = '2021-06-14 23:00'
    
    start_idx = fcst_df[fcst_df['Forecast_time']==start].index[0]
    end_idx = fcst_df[fcst_df['Forecast_time']==end].index[0]
    
    test_df = fcst_df.loc[start_idx:end_idx, :].copy()
    
    test_df['date'] = test_df['Forecast_time'].str.split(' ').str[0]
    test_df['hour'] = test_df['Forecast_time'].str.split(' ').str[1].str.split(':').str[0].astype(int)
    
    test_df['year'] = test_df['date'].str.split('-').str[0].astype(int)
    test_df['month'] = test_df['date'].str.split('-').str[1].astype(int)
    test_df['day'] = test_df['date'].str.split('-').str[2].astype(int)
    
    test_df = test_df[['year', 'month', 'day', 'hour', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud']]
    
    test_x = test_df.to_numpy()
    
    return test_x

In [33]:
dangjin_test = test_datast(dangjin)
ulsan_test = test_datast(ulsan)

## 각 발전소 발전량 추론

In [34]:
submission = pd.read_csv('./data/sample_submission.csv')

In [57]:
model =  joblib.load('./dangjin_floating/dangjin_floating_{0}.pkl'.format(i+1))

In [65]:
lgb1_models = {}
for i in range(10):
    model =  joblib.load('./dangjin_floating/dangjin_floating_{0}.pkl'.format(i+1))
    lgb1_models[i] = model

In [ ]:
lgb1_models

In [69]:
dangjin_floating_pred = submission.iloc[:24, 1]

for fold in range(10):
    dangjin_floating_pred += lgb1_models[fold].predict(dangjin_test)/10
    #dangjin_floating_pred += cat1_models[fold].predict(dangjin_test)/20


#dangjin_floating_pred += dangjin_floating_model.predict(dangjin_test) 
submission.iloc[24*28+24*(day-1):24*28+24*day, 1] = dangjin_floating_pred.to_list()

In [68]:
cat2_models = {}
for i in range(10):
    model =  joblib.load('./dangjin_warehouse/dangjin_warehouse{0}.pkl'.format(i+1))
    cat2_models[i] = model
cat2_models

{0: <catboost.core.CatBoostRegressor at 0x7f7029a8f350>,
 1: <catboost.core.CatBoostRegressor at 0x7f7029eaba50>,
 2: <catboost.core.CatBoostRegressor at 0x7f7029a8fc10>,
 3: <catboost.core.CatBoostRegressor at 0x7f702a0c5f90>,
 4: <catboost.core.CatBoostRegressor at 0x7f7080f244d0>,
 5: <catboost.core.CatBoostRegressor at 0x7f70805b5dd0>,
 6: <catboost.core.CatBoostRegressor at 0x7f7029f10c50>,
 7: <catboost.core.CatBoostRegressor at 0x7f7029f107d0>,
 8: <catboost.core.CatBoostRegressor at 0x7f7029f10510>,
 9: <catboost.core.CatBoostRegressor at 0x7f7029e28f10>}

In [70]:
dangjin_warehouse_pred = submission.iloc[:24, 2]

for fold in range(10):
    
    dangjin_warehouse_pred += cat2_models[fold].predict(dangjin_test)/10
    
#dangjin_warehouse_pred += cat2.predict(dangjin_test)
submission.iloc[24*28+24*(day-1):24*28+24*day, 2] = dangjin_warehouse_pred.to_list()

In [71]:
cat3_models = {}
for i in range(10):
    model =  joblib.load('./dangjin/dangjin{0}.pkl'.format(i+1))
    cat3_models[i] = model
cat3_models

{0: <catboost.core.CatBoostRegressor at 0x7f707ff3dfd0>,
 1: <catboost.core.CatBoostRegressor at 0x7f7029eab710>,
 2: <catboost.core.CatBoostRegressor at 0x7f7029ec6190>,
 3: <catboost.core.CatBoostRegressor at 0x7f7080bf9f10>,
 4: <catboost.core.CatBoostRegressor at 0x7f7029ec6850>,
 5: <catboost.core.CatBoostRegressor at 0x7f7029e5cc90>,
 6: <catboost.core.CatBoostRegressor at 0x7f7029e93450>,
 7: <catboost.core.CatBoostRegressor at 0x7f70298e6bd0>,
 8: <catboost.core.CatBoostRegressor at 0x7f7029e5cb10>,
 9: <catboost.core.CatBoostRegressor at 0x7f7029eccd50>}

In [72]:
dangjin_pred = submission.iloc[:24, 3]

for fold in range(10):

    dangjin_pred += cat3_models[fold].predict(dangjin_test)/10

#dangjin_pred += cat3.predict(dangjin_test)
submission.iloc[24*28+24*(day-1):24*28+24*day, 3] = dangjin_pred.to_list()

In [73]:
cat4_models = {}
for i in range(10):
    model =  joblib.load('./ulsan/ulsan{0}.pkl'.format(i+1))
    cat4_models[i] = model
cat4_models

{0: <catboost.core.CatBoostRegressor at 0x7f70298e6d50>,
 1: <catboost.core.CatBoostRegressor at 0x7f7080aa7150>,
 2: <catboost.core.CatBoostRegressor at 0x7f7080aa70d0>,
 3: <catboost.core.CatBoostRegressor at 0x7f707ff3d250>,
 4: <catboost.core.CatBoostRegressor at 0x7f70805b5ad0>,
 5: <catboost.core.CatBoostRegressor at 0x7f70805b5790>,
 6: <catboost.core.CatBoostRegressor at 0x7f70805bab10>,
 7: <catboost.core.CatBoostRegressor at 0x7f70a21be1d0>,
 8: <catboost.core.CatBoostRegressor at 0x7f70805ba390>,
 9: <catboost.core.CatBoostRegressor at 0x7f70805ba350>}

In [74]:
ulsan_pred = submission.iloc[:24, 4]


for fold in range(10):
    #ulsan_pred += lgb4_models[fold].predict(ulsan_test)/20
    ulsan_pred += cat4_models[fold].predict(ulsan_test)/10
    

#ulsan_pred += cat4.predict(ulsan_test)    
submission.iloc[24*28+24*(day-1):24*28+24*day, 4] = ulsan_pred.to_list()

In [75]:
submission.iloc[24*28+24*(day-1):24*28+24*day,:]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
792,2021-06-14 01:00:00,29.293530,14.466465,33.433374,-3.152566
793,2021-06-14 02:00:00,29.293530,14.447154,33.339689,-4.158468
794,2021-06-14 03:00:00,27.899728,14.421130,32.728642,-4.816339
795,2021-06-14 04:00:00,4.261173,13.186454,20.088906,-4.823425
796,2021-06-14 05:00:00,4.637319,12.111895,20.426821,-5.834965
797,2021-06-14 06:00:00,3.796286,9.761324,17.193279,-3.309316
798,2021-06-14 07:00:00,17.617929,12.481400,15.200005,1.525176
799,2021-06-14 08:00:00,61.469259,47.364428,48.791473,15.424634
800,2021-06-14 09:00:00,137.160220,124.386809,161.222710,42.143650
801,2021-06-14 10:00:00,244.127949,208.257934,279.167963,52.663561


In [41]:
submission.iloc[24*28+24*(day-1):24*28+24*day,:]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
792,2021-06-14 01:00:00,29.293530,14.466465,33.433374,-3.152566
793,2021-06-14 02:00:00,29.293530,14.447154,33.339689,-4.158468
794,2021-06-14 03:00:00,27.899728,14.421130,32.728642,-4.816339
795,2021-06-14 04:00:00,4.261173,13.186454,20.088906,-4.823425
796,2021-06-14 05:00:00,4.637319,12.111895,20.426821,-5.834965
797,2021-06-14 06:00:00,3.796286,9.761324,17.193279,-3.309316
798,2021-06-14 07:00:00,17.617929,12.481400,15.200005,1.525176
799,2021-06-14 08:00:00,61.469259,47.364428,48.791473,15.424634
800,2021-06-14 09:00:00,137.160220,124.386809,161.222710,42.143650
801,2021-06-14 10:00:00,244.127949,208.257934,279.167963,52.663561


In [42]:
submission.to_csv(fcst_day+'.csv',index=False)

## 테스트 데이터 전처리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 제출

In [ ]:
def test_datast(fcst_df):
    start = '2021-02-01 00:00:00'
    end = '2021-02-28 23:00:00'
    
    start_idx = fcst_df[fcst_df['Forecast_time']==start].index[0]
    end_idx = fcst_df[fcst_df['Forecast_time']==end].index[0]
    
    test_df = fcst_df.loc[start_idx:end_idx, :].copy()
    
    test_df['date'] = test_df['Forecast_time'].str.split(' ').str[0]
    test_df['hour'] = test_df['Forecast_time'].str.split(' ').str[1].str.split(':').str[0].astype(int)
    
    test_df['year'] = test_df['date'].str.split('-').str[0].astype(int)
    test_df['month'] = test_df['date'].str.split('-').str[1].astype(int)
    test_df['day'] = test_df['date'].str.split('-').str[2].astype(int)
    
    test_df = test_df[['year', 'month', 'day', 'hour', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud']]
    
    test_x = test_df.to_numpy()
    
    return test_x

In [ ]:
dangjin_test = test_datast(dangjin_interpolated)
ulsan_test = test_datast(ulsan_interpolated)

In [ ]:
submission = pd.read_csv('./data/sample_submission.csv')

In [ ]:
dangjin_floating_pred = submission.iloc[:24*28, 1]

for fold in range(10):
    dangjin_floating_pred += lgb1_models[fold].predict(dangjin_test)/10
    

#dangjin_floating_pred = dangjin_floating_model.predict(dangjin_test)
submission.iloc[:24*28, 1] = dangjin_floating_pred

In [ ]:
dangjin_warehouse_pred = submission.iloc[:24*28, 2]

for fold in range(10):
    
    dangjin_warehouse_pred += cat2_models[fold].predict(dangjin_test)/10
#dangjin_warehouse_pred = cat2.predict(dangjin_test)
submission.iloc[:24*28, 2] = dangjin_warehouse_pred

In [ ]:
dangjin_pred = submission.iloc[:24*28, 2]

for fold in range(5):
    #dangjin_pred += lgb3_models[fold].predict(dangjin_test)/20
    dangjin_pred += cat3_models[fold].predict(dangjin_test)/5
#dangjin_pred = cat3.predict(dangjin_test)
submission.iloc[:24*28, 3] = dangjin_pred

In [ ]:
ulsan_pred = submission.iloc[:24*28, 4]

'''for fold in range(10):
    #ulsan_pred += lgb4_models[fold].predict(ulsan_test)/20
    ulsan_pred += cat4_models[fold].predict(ulsan_test)/10'''
ulsan_pred = cat4.predict(ulsan_test)
ulsan_pred = submission.iloc[:24*28, 4] = ulsan_pred

In [ ]:
submission.to_csv('final2222.csv', index=False)

In [ ]:
submission[:20]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,-14.030844,-3.929230,-3.929230,-3.411006
1,2021-02-01 02:00:00,-10.785423,-0.544544,-0.544544,-1.206727
2,2021-02-01 03:00:00,-13.939630,-1.851262,-1.851262,0.020413
3,2021-02-01 04:00:00,-16.172595,-7.913679,-7.913679,0.057985
4,2021-02-01 05:00:00,-13.610172,-7.420329,-7.420329,0.052356
5,2021-02-01 06:00:00,-14.192924,-10.949913,-10.949913,0.508380
6,2021-02-01 07:00:00,-17.096378,-15.462109,-15.462109,2.761307
7,2021-02-01 08:00:00,-6.982919,-9.544430,-9.544430,6.430773
8,2021-02-01 09:00:00,17.643115,49.958906,49.958906,18.850496
9,2021-02-01 10:00:00,21.132810,77.033430,77.033430,35.471467


In [ ]:
submission.iloc[:24*28, 1] = dangjin_floating_pred
submission.iloc[:24*28, 2] = dangjin_warehouse_pred
submission.iloc[:24*28, 3] = dangjin_pred
submission.iloc[:24*28, 4] = ulsan_pred

In [ ]:
submission2 = pd.read_csv('./data/sample_submission.csv')
submission2.iloc[:24*28, 1] = dangjin_floating_pred
submission2.iloc[:24*28, 2] = dangjin_warehouse_pred
submission2.iloc[:24*28, 3] = dangjin_pred
submission2.iloc[:24*28, 4] = ulsan_pred

In [ ]:
submission[['dangjin_floating','dangjin_warehouse','dangjin','ulsan']] = (submission[['dangjin_floating','dangjin_warehouse','dangjin','ulsan']] + submission2[['dangjin_floating','dangjin_warehouse','dangjin','ulsan']]) / 2

In [ ]:
submission

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,-6.904098,-0.891064,0.659742,-1.984275
1,2021-02-01 02:00:00,-6.691012,-0.693226,0.816881,-1.073283
2,2021-02-01 03:00:00,-6.429722,-1.361885,0.809705,-0.190118
3,2021-02-01 04:00:00,-5.708050,-1.485486,0.548285,-0.191640
4,2021-02-01 05:00:00,-3.146568,-1.651132,0.530432,-0.169720
...,...,...,...,...,...
1387,2021-07-08 20:00:00,0.000000,0.000000,0.000000,0.000000
1388,2021-07-08 21:00:00,0.000000,0.000000,0.000000,0.000000
1389,2021-07-08 22:00:00,0.000000,0.000000,0.000000,0.000000
1390,2021-07-08 23:00:00,0.000000,0.000000,0.000000,0.000000


In [ ]:
submission.to_csv('lgb+cat.csv', index=False)